In [2]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r"C:\Users\LENOVO\CV\Automated-Fabric-Defect-Inspection\textile_defect_data.hdF5"
imgs = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]

# Define the displacement vector (1, 0) for horizontal pairs
displacement = (1, 0)

# Define the number of histogram bins
num_bins = 256

# Define the feature vector
feature_vector = []

for img in imgs:

    # Calculate the co-occurrence matrix using OpenCV
    hist = cv2.calcHist([img], [0], None, [num_bins], [0, num_bins])
    hist_transpose = np.transpose(hist)
    co_matrix = cv2.calcHist([img, img], [0, 1], None, [num_bins, num_bins], [0, num_bins, 0, num_bins], accumulate=False)

    # Normalize the co-occurrence matrix
    co_matrix_normalized = cv2.normalize(co_matrix, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Calculate contrast
    contrast = np.sum((co_matrix_normalized * np.arange(num_bins)[:, np.newaxis] - np.mean(co_matrix_normalized)) ** 2)

    # Calculate energy
    energy = np.sum(co_matrix_normalized ** 2)

    # Calculate homogeneity
    homogeneity = np.sum(co_matrix_normalized / (1 + np.abs(np.arange(num_bins)[:, np.newaxis] - np.arange(num_bins))))

    # Calculate correlation
    x, y = np.meshgrid(np.arange(num_bins), np.arange(num_bins))
    correlation = np.sum((co_matrix_normalized * (x - np.mean(x)) * (y - np.mean(y))) / (np.std(x) * np.std(y)))

    # Add the feature vector for the current image to the feature vector list
    feature_vector.append([contrast, energy, homogeneity, correlation])

# Convert the feature vector list to a numpy array
feature_vector = np.array(feature_vector)

# Print the feature vector for the first image
print(len(feature_vector))

# Principle components
pca=PCA()
principle_components=pca.fit_transform(feature_vector)

# svm
x_train,x_test,y_train,y_test=train_test_split(principle_components,labels,test_size=0.25,random_state=42)
classifier=SVC(random_state=42)
classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)

print("Accuracy Score",accuracy_score(y_test,y_pred))


300
Accuracy Score 0.4666666666666667
